In [ ]:
%load_ext autoreload
%autoreload 2

import os
import shutil

try:
  import google.colab
  os.system("git clone https://github.com/matt-nann/AuthenticCursor.git")
  try:
    shutil.copytree("AuthenticCursor/src", "src")
  except:
    shutil.rmtree("src")
    shutil.copytree("AuthenticCursor/src", "src")
  try:
    shutil.copy("AuthenticCursor/requirementsGAN.txt", "requirementsGAN.txt")
  except:
    shutil.rmtree("requirementsGAN.txt")
    shutil.copy("AuthenticCursor/requirementsGAN.txt", "requirementsGAN.txt")
  # remove conflicting dependencies with google colab preinstalled libraries
  with open("requirementsGAN.txt", "r") as f:
    lines = f.readlines()
    with open("requirementsGAN.txt", "w") as f:
      for line in lines:
        if "numpy" not in line and 'pillow' not in line:
          f.write(line)
  os.system("pip install -r requirementsGAN.txt")
  shutil.rmtree("AuthenticCursor")
  # installing and logging into weights and biases
  os.system("pip install wandb")
  os.system("wandb login")
except Exception as e:
  print(e)

import torch
import wandb # will be prompted for API key in google colab

# check for cuda
if not torch.cuda.is_available():
  raise Exception("CUDA not available")

In [ ]:
from src.mouseGAN.dataProcessing import MouseGAN_Data
from src.mouseGAN.dataset import getDataloader, visuallyVertifyDataloader

USE_FAKE_DATA = True
SAVE_FAKE_DATA = False
RELOAD_FAKE_DATA = True
TRAIN_TEST_SPLIT = 0.8
dataset = MouseGAN_Data(USE_FAKE_DATA=USE_FAKE_DATA, TRAIN_TEST_SPLIT=TRAIN_TEST_SPLIT, 
                        equal_length=False)

SAMPLES = 10000
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False
if USE_FAKE_DATA:
    if RELOAD_FAKE_DATA:
        datasetProperties = dict(low_radius = 10, high_radius = 12,
                                        max_width = 2, min_width = 1,
                                        max_height = 2, min_height = 1,)
        dataset.createFakeWindMouseDataset(save=SAVE_FAKE_DATA, samples=SAMPLES,
                                            **datasetProperties)
        dataset.saveFakeDataProperties(datasetProperties)
    else:
        dataset.loadFakeWindMouseData()
else:
    df_moves, df_trajectory = dataset.collectRawMouseTrajectories()

In [ ]:
import time
s_time = time.time()
train_trajs, train_targets, test_trajs, test_targets = dataset.processMouseData(SHOW_ALL=False)
print(f"Time to process data: {time.time() - s_time} seconds")

In [100]:
from src.mouseGAN.LR_schedulers import *
from src.mouseGAN.model_config import Config, LR_SCHEDULERS, LOSS_FUNC, \
    C_MiniBatchDisc, C_Discriminator, C_Generator, C_EMA_Plateua_Sch, \
    C_Step_Sch, C_LossGap_Sch
from src.mouseGAN.models import MouseGAN
from src.mouseGAN.experimentTracker import initialize_wandb

# IN_COLAB = True
LOAD_PRETRAINED = True
BATCH_SIZE = 256
num_epochs = 1000
num_feats = train_trajs[0].shape[1]
latent_dim = 20
num_target_feats = 4 # width, height, start_x, start_y
numBatches = len(train_trajs)//BATCH_SIZE
MAX_SEQ_LEN = max([len(traj) for traj in train_trajs + test_trajs])

D_config = C_Discriminator(lr=0.01, bidirectional=True, hidden_units=128, 
                            num_lstm_layers=1, 
                            useEndDeviationLoss=True,
                            gradient_maxNorm = 1.0,
                            spectral_norm = True)
G_config = C_Generator(lr=0.0002, hidden_units=128, num_lstm_layers=3, drop_prob=0.4,
                # layer_normalization = True,
                residual_connections = True,
                gradient_maxNorm = 1.0,
                useSeqLengthLoss=True,
                lengthLossWeight = 0.25,
                useOutsideTargetLoss=True,
                outsideTargetLossWeight = 1,
                usePathLengthLoss=True,
                pathLengthLossWeight = 0.25)

# D_sch_config = C_Step_Sch(2, 0.5)
D_sch_config = C_LossGap_Sch(cooldown=int(numBatches/4), lr_shrinkMin=0.1, lr_growthMax=2.0, 
                            discLossDecay=0.8, lr_max = D_config.lr, lr_min = 0.0001, restart_after=None)
# G_sch_config = C_Step_Sch(2, 0.5)
# G_sch_config = C_EMA_Plateua_Sch(patience=BATCH_SIZE, cooldown=int(BATCH_SIZE/8), factor=0.5, ema_alpha=0.4)

config = Config(num_epochs, BATCH_SIZE, num_feats, latent_dim, num_target_feats, MAX_SEQ_LEN,
                discriminator=D_config, generator=G_config, 
                D_lr_scheduler=D_sch_config, #G_lr_scheduler=G_sch_config,
                locationMSELoss = True)

## verifying the mean trajectory is centered around zero (even class distribution)
# dataset.plotMeanPath()
trainLoader = getDataloader(train_trajs, train_targets, config.BATCH_SIZE)
testLoader = getDataloader(test_trajs, test_targets, config.BATCH_SIZE)

visuallyVertifyDataloader(trainLoader, dataset, showNumBatches=1)
 
# if IN_COLAB:
run = initialize_wandb(config, tempProjectName='mouseGAN_debug')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gan = MouseGAN(dataset, trainLoader, testLoader, device, config, IN_COLAB=IN_COLAB, verbose=True, printBatch=True)
if LOAD_PRETRAINED:
    gan.loadPretrained(startingEpoch=300)

gan.visualTrainingVerfication(samples=10)

# print(gan.discriminator)
# print(gan.generator)

# gan.find_learning_rates_for_GAN()
gan.train(modelSaveInterval=20, catchErrors=False, visualCheckInterval=5)
# if IN_COLAB:
#     wandb.finish()

D_lr,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
d_fake_logits,▇▇█▅▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▅▂▄▄▂▂▄▂▁▁▁▄▄▃▄▃▃▁
d_loss,▁▄█▆▆▇▅▅▅▄▅▆▅▅▄▄▃▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▃▂▂▂▂▂
d_loss_base,▇▅█▄▄█▃▄▃▂▃▆▂▃▂▂▁▂▁▁▁▁▄▁▂▂▁▁▂▁▃▁▁▁▃▁▁▁▁▃
d_loss_fake,▆▅█▃▅▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▃▁▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁
d_loss_fake_dev,▁▅█▇▇▇▇▆▆▆▆▆▆▆▅▅▅▄▄▄▄▄▃▄▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▂
d_loss_real,▄▂▂▂▂█▂▄▂▂▂▇▁▃▁▂▁▂▂▁▁▁▃▁▁▁▁▁▁▂▄▁▁▁▄▁▁▁▁▄
d_loss_real_dev,▁▃▂▄▃▃▃▂▃▃▃▃▄▄▅▆▆▆▆▆▇▆▆▇█▇▇▇▇▅▆▆▆▇▇▆▆▆▆▇
d_real_logits,▃▆█▇▆▁▆▃▅▆▅▁▆▃▅▄▇██▅▅▇▃▆▇▅▆▅▅▇▃▅▆▆▂▇▇▆▇▃
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
epochTime,▃▃▃▂▂▄▄▄▄▄▅▅▅▄▄▃▆▇▅▆▆▁▅▇▅▃▃▄▃▄▄▃▅▇▅▅▆▆▅█


Loaded generator model: /Users/mnann/Documents/Code/AuthenticCursor/data/local/ganModels/g300.pt
Loaded discriminator model: /Users/mnann/Documents/Code/AuthenticCursor/data/local/ganModels/d300.pt
Starting from epoch 300


/Users/mnann/Documents/Code/AuthenticCursor/venvDev/lib/python3.8/site-packages/torch/cuda/amp/grad_scaler.py:120: UserWarning:

torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.



wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


	Batch 1/32, d_loss = 1.422, g_loss = 10.830
	Batch 4/32, d_loss = 2.171, g_loss = 8.931
	Batch 7/32, d_loss = 1.825, g_loss = 7.092
	Batch 10/32, d_loss = 1.411, g_loss = 3.082
	Batch 13/32, d_loss = 2.551, g_loss = 3.587
	Batch 16/32, d_loss = 1.597, g_loss = 5.680
	Batch 19/32, d_loss = 1.543, g_loss = 5.949
	Batch 22/32, d_loss = 1.433, g_loss = 5.490
	Batch 25/32, d_loss = 1.350, g_loss = 5.370
	Batch 28/32, d_loss = 1.424, g_loss = 5.511
	Batch 31/32, d_loss = 1.523, g_loss = 5.773


...

{'epoch': 390, 'd_loss': '1.71798', 'g_loss': '5.94268', 'epochTime': '6.70179', 'val_accuracy': '0.93140', 'val_d_loss': '1.81216', 'val_g_loss': '6.45127'}
	Batch 1/32, d_loss = 1.713, g_loss = 6.953
	Batch 4/32, d_loss = 1.680, g_loss = 5.347
	Batch 7/32, d_loss = 1.587, g_loss = 7.237
	Batch 10/32, d_loss = 1.592, g_loss = 7.214
	Batch 13/32, d_loss = 1.722, g_loss = 7.255
	Batch 16/32, d_loss = 1.612, g_loss = 5.628
	Batch 19/32, d_loss = 1.807, g_loss = 6.903
	Batch 22/32, d_loss = 1.630, g_loss = 6.875
	Batch 25/32, d_loss = 1.530, g_loss = 7.014
	Batch 28/32, d_loss = 1.588, g_loss = 6.747
	Batch 31/32, d_loss = 1.649, g_loss = 5.271
{'epoch': 391, 'd_loss': '1.71743', 'g_loss': '6.38787', 'epochTime': '6.62427', 'val_accuracy': '0.92773', 'val_d_loss': '1.78720', 'val_g_loss': '5.66523'}
	Batch 1/32, d_loss = 1.498, g_loss = 6.237
	Batch 4/32, d_loss = 1.815, g_loss = 4.757
	Batch 7/32, d_loss = 1.841, g_loss = 4.744
	Batch 10/32, d_loss = 1.744, g_loss = 4.612
	Batch 13/32, d

{'epoch': 395, 'd_loss': '1.65265', 'g_loss': '5.20989', 'epochTime': '7.00441', 'val_accuracy': '0.97656', 'val_d_loss': '1.56342', 'val_g_loss': '4.68018'}
	Batch 1/32, d_loss = 1.698, g_loss = 4.583
	Batch 4/32, d_loss = 1.452, g_loss = 4.149
	Batch 7/32, d_loss = 1.670, g_loss = 4.178
	Batch 10/32, d_loss = 1.555, g_loss = 3.876
	Batch 13/32, d_loss = 1.971, g_loss = 3.914
	Batch 16/32, d_loss = 1.503, g_loss = 4.079
	Batch 19/32, d_loss = 1.467, g_loss = 4.093
	Batch 22/32, d_loss = 1.420, g_loss = 4.460
	Batch 25/32, d_loss = 1.753, g_loss = 4.670
	Batch 28/32, d_loss = 1.505, g_loss = 5.122
	Batch 31/32, d_loss = 1.419, g_loss = 5.538
{'epoch': 396, 'd_loss': '1.58009', 'g_loss': '4.97163', 'epochTime': '7.89688', 'val_accuracy': '0.94897', 'val_d_loss': '1.58362', 'val_g_loss': '5.39397'}
	Batch 1/32, d_loss = 1.548, g_loss = 5.561
	Batch 4/32, d_loss = 1.422, g_loss = 5.843
	Batch 7/32, d_loss = 1.850, g_loss = 5.642
	Batch 10/32, d_loss = 1.504, g_loss = 5.571
	Batch 13/32, d

{'epoch': 400, 'd_loss': '1.73914', 'g_loss': '5.58279', 'epochTime': '7.44680', 'val_accuracy': '0.93433', 'val_d_loss': '1.61324', 'val_g_loss': '5.22236'}
	Batch 1/32, d_loss = 1.539, g_loss = 5.762
	Batch 4/32, d_loss = 2.125, g_loss = 5.268
	Batch 7/32, d_loss = 1.553, g_loss = 5.897
	Batch 10/32, d_loss = 1.549, g_loss = 5.305
	Batch 13/32, d_loss = 1.625, g_loss = 6.107
	Batch 16/32, d_loss = 1.998, g_loss = 6.274
	Batch 19/32, d_loss = 2.020, g_loss = 6.564
	Batch 22/32, d_loss = 1.650, g_loss = 6.642
	Batch 25/32, d_loss = 2.591, g_loss = 6.802
	Batch 28/32, d_loss = 2.222, g_loss = 7.379
	Batch 31/32, d_loss = 2.178, g_loss = 7.145
{'epoch': 401, 'd_loss': '1.84064', 'g_loss': '6.19993', 'epochTime': '7.08568', 'val_accuracy': '0.97119', 'val_d_loss': '2.01168', 'val_g_loss': '6.47158'}
	Batch 1/32, d_loss = 1.734, g_loss = 6.982
	Batch 4/32, d_loss = 2.156, g_loss = 6.553
	Batch 7/32, d_loss = 2.044, g_loss = 6.581
	Batch 10/32, d_loss = 1.809, g_loss = 6.655
	Batch 13/32, d

{'epoch': 405, 'd_loss': '2.00339', 'g_loss': '7.35216', 'epochTime': '7.52369', 'val_accuracy': '0.94043', 'val_d_loss': '1.84729', 'val_g_loss': '5.96899'}
	Batch 1/32, d_loss = 2.098, g_loss = 6.542
	Batch 4/32, d_loss = 1.915, g_loss = 6.422
	Batch 7/32, d_loss = 1.914, g_loss = 7.050
	Batch 10/32, d_loss = 2.154, g_loss = 4.399
	Batch 13/32, d_loss = 2.165, g_loss = 6.489
	Batch 16/32, d_loss = 2.152, g_loss = 6.606
	Batch 19/32, d_loss = 2.146, g_loss = 6.899
	Batch 22/32, d_loss = 1.898, g_loss = 7.589
	Batch 25/32, d_loss = 2.269, g_loss = 7.486


KeyboardInterrupt: 

In [98]:
gan.visualTrainingVerfication(samples=50)

In [ ]:
gan.visualTrainingVerfication(samples=10)

In [ ]:
gan.visualTrainingVerfication(samples=10)

In [ ]:
gan.train(modelSaveInterval=3, catchErrors=False)

In [ ]:
# gan.save_models('final')
gan.loadPretrained(startingEpoch=99)

In [ ]:
for epoch in ['final']:
    gan.loadPretrained(startingEpoch=epoch)
    gan.visualTrainingVerfication()

In [ ]:
a = torch.tensor(1.5)
a.requires_grad_()
b = torch.greater(a, 1.0)
# b = torch.round(a)
b.backward()
a.grad